# Data Generation (Supervised Learning)

## Import Packages

In [1]:
import numpy as np
import scipy
from scipy.stats import norm
import pandapower as pp
import pandas as pd
import pandapower.networks
import tensorflow as tf
import csv
import torch

## Input Data

In [2]:
case = pp.networks.case30()
number_of_bus = case.bus.shape[0]
baseMVA = 100

initial_voltage = 1
initial_delta = 0

number_of_data = 9000       # Including data for both Training and Testing
number_of_training_data = 8000
number_of_test_data = np.subtract(number_of_data,number_of_training_data)

load = case.load.iloc[:,6:8]
load_bus_number = case.load.iloc[:,0]

Pgenerators = np.zeros((number_of_bus,1))
Pgenerators[case.gen.iloc[:,0],0] = -case.gen.iloc[:,4]

PQload = np.zeros((number_of_bus,2))
PQload[case.load.iloc[:,0],:] = case.load.iloc[:,6:8]



### R X  PL QL PG initial_voltage initial_delta

Data_form = np.hstack((np.array(case.bus_geodata.iloc[:,0:2]),\
                       PQload,Pgenerators,\
                      np.full((number_of_bus,1),initial_voltage),\
                     np.full((number_of_bus,1),initial_delta)))

## Roulette Wheel Algorithm Using Normal-PDF to Generate Scenarios

In [113]:
Scenario = np.zeros([number_of_data,Data_form.shape[0],Data_form.shape[1]])

for i in range(number_of_data):
    
    Scenario[i,:,:] = Data_form
    Scenario[i,load_bus_number,2:4] = norm.ppf(np.random.rand(load.shape[0],1),\
                                                              load,np.multiply(0.1,load))

Training_Data = Scenario[0:number_of_training_data,:,:]
Test_Data = Scenario[number_of_training_data:,:,:]

## Implement Power Flow to Get Labels

In [114]:
Label = np.zeros((number_of_data,number_of_bus,2))

for i in range(number_of_data):
    
    case.load.iloc[:,6:8] = Scenario[i,case.load.iloc[:,0],2:4]
    
    pp.runpp(case, algorithm='nr', calculate_voltage_angles='auto', init='auto',\
                  max_iteration='auto', tolerance_mva=1e-08, trafo_model='t',\
                  trafo_loading='current', enforce_q_lims=False,\
                  check_connectivity=True, voltage_depend_loads=True,\
                 consider_line_temperature=False)
    
    Label[i,:,:] = case.res_bus.iloc[:,0:2]
    
Training_Label = Label[0:number_of_training_data,:,:]
Test_Label = Label[number_of_training_data:,:,:]


numba cannot be imported and numba functions are disabled.
Probably the execution is slow.
Please install numba to gain a massive speedup.
(or if you prefer slow execution, set the flag numba=False to avoid this warning!)

numba cannot be imported and numba functions are disabled.
Probably the execution is slow.
Please install numba to gain a massive speedup.
(or if you prefer slow execution, set the flag numba=False to avoid this warning!)

numba cannot be imported and numba functions are disabled.
Probably the execution is slow.
Please install numba to gain a massive speedup.
(or if you prefer slow execution, set the flag numba=False to avoid this warning!)

numba cannot be imported and numba functions are disabled.
Probably the execution is slow.
Please install numba to gain a massive speedup.
(or if you prefer slow execution, set the flag numba=False to avoid this warning!)

numba cannot be imported and numba functions are disabled.
Probably the execution is slow.
Please install nu

## Exporting Data to CSV files

In [115]:
Final_Training_Data = pd.DataFrame(Training_Data.reshape(number_of_training_data,-1))
Final_Training_Label  = pd.DataFrame(Training_Label.reshape(number_of_training_data,-1))
Final_Training_Data.to_csv("C:/Users/adria/OneDrive/Desktop/DataGeneration/train_data_30bus.csv",index = False)
Final_Training_Label.to_csv("C:/Users/adria/OneDrive/Desktop/DataGeneration/train_label_30bus.csv",index = False)

Final_Test_Data   = pd.DataFrame(Test_Data.reshape(number_of_test_data,-1))
Final_Test_Label  = pd.DataFrame(Test_Label.reshape(number_of_test_data,-1))
Final_Test_Data.to_csv("C:/Users/adria/OneDrive/Desktop/DataGeneration/test_data_30bus.csv",index = False)
Final_Test_Label.to_csv("C:/Users/adria/OneDrive/Desktop/DataGeneration/test_label_30bus.csv",index = False)


